In [ ]:
import os
import re
import zipfile
import smtplib
import argparse
import getpass
import sys
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email import encoders

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--mailid', help='send to')               #senders mail id
parser.add_argument('--attachment', help='attachment file')
args = parser.parse_args()

In [ ]:
from_address = " "       #Enter emailid
password = getpass.getpass('Password for the email '+ from_address +' : ')
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
try:
    server.login(from_address, password)
except:
    print('Invalid password')
    sys.exit(1)

sender = from_address
to_address = args.mailid



In [ ]:
path_of_files = args.attachment    #path of the folder    "/home/harihara/Desktop/hari"
name_of_zip_file = " "  # Give a name to the zip file you want to send
subject = " "           # Subject to email
text = " "              # Enter text you want to send

In [ ]:
# Create a Zip file
def zip(src, dst):
    zf = zipfile.ZipFile("%s.zip" % (dst), "w")
    src = os.path.abspath(src)
    for d, s, f in os.walk(src):
        for n in f:
            if re.match(r"^.*[.](py|ipynb|txt)$", n):               #type of file you need to add
                abs_name = os.path.abspath(os.path.join(d, n))
                arc_name = abs_name[len(src) + 1:]
                zf.write(abs_name, arc_name)
    zf.close()

In [ ]:
# Send gmail
def sendMsg():
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = from_address

    msg['To'] = to_address
    msg.attach(MIMEText(text,'plain'))

    part = MIMEBase("application", "octet-stream")
    part.set_payload(open(name_of_zip_file + ".zip", "rb").read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", "attachment; filename=\"%s.zip\"" % (name_of_zip_file))
    msg.attach(part)
    
    server.sendmail(sender, to_address, msg.as_string())
    server.quit()

In [ ]:
zip(path_of_files,name_of_zip_file)
sendMsg()